In [ ]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
import math

# Load the tads+genes files for human and axolotl separately 
# Format:
#   chr     start    end      gene1;gene2;gene3
def loadTads(strFilename):
    tads = dict()
    with open(strFilename, 'r') as hFile:
        for strLine in hFile.readlines():
            chrID, start, end, genes = strLine.strip().split('\t')
            tads[f'{chrID}:{start}-{end}'] = genes.split(';')
    return (tads)


def getTADsforGene(tads):
    genes = dict()
    for coord in tads:
        for geneID in tads[coord]:
            if genes.get(geneID):
                genes[geneID].append(coord)
            else:
                genes[geneID] = [coord]
    return (genes)


def findMatchingTAD(queryGenes, targetGenes):
    tadlist = []
    for geneID in queryGenes:
        tads = targetGenes.get(geneID)
        if tads:
            for tad in tads:
                tadlist.append(tad)
    # Now find the most common TAD
    nMax = 0
    best = None
    for tad in tadlist:
        n = tadlist.count(tad)
        if n > nMax:
            nMax = n
            best = tad
    return(best) 
    

####### Main #######
ambMex_tads = loadTads('/groups/tanaka/Projects/axolotl-genome/AmexG_v6.0/AmexG_v6.0_DD/work/manuscript/compare_TADs/ambMex60DD.putative.tads+genes.bed')
print(f'Loaded {len(ambMex_tads)} axolotl TADs')

hg19_tads = loadTads('/groups/tanaka/Projects/axolotl-genome/AmexG_v6.0/AmexG_v6.0_DD/work/manuscript/compare_TADs/hg19.tads+genes')
print(f'Loaded {len(hg19_tads)} human TADs')

# Iterate over the human TADs and find the axolotl TAD with the most genes present
with open('/groups/tanaka/Projects/axolotl-genome/AmexG_v6.0/AmexG_v6.0_DD/work/manuscript/compare_TADs/hg19_vs_ambMex.matches', 'w') as hFile:
    ambMex_genes = getTADsforGene(ambMex_tads)
    for tadCoord in hg19_tads:
        bestMatch = findMatchingTAD(hg19_tads[tadCoord], ambMex_genes)
        print(f'Human: {tadCoord}\tAmbMex: {bestMatch}', file=hFile)
    print("Done")

In [ ]:
## Paco's block
import re
File = open("/groups/tanaka/Projects/axolotl-genome/AmexG_v6.0/AmexG_v6.0_DD/work/manuscript/compare_TADs/ambMex60DD.putative.tads+genes.bed",'r')

Hum_TADs = dict()
Amex_TADs = dict()

for x in File:
    x = x.split("\t")
    Amex_TADs["-".join(x[0:3])] = re.split(',|\||;',x[3][:-2]) 
    
File = open("/groups/tanaka/Projects/axolotl-genome/AmexG_v6.0/AmexG_v6.0_DD/work/manuscript/compare_TADs/hg19.tads+genes")

for x in File:
    x = x.split("\t")
    Hum_TADs["-".join(x[0:3])] = re.split(',|\||;',x[3][:-2]) 

In [ ]:
## Pacos compare tads
Hum_TAD_Compare = dict()
for i in Hum_TADs:
    for j in Amex_TADs:
        Counter = 0
        for k in Hum_TADs[i]:
            if k in Amex_TADs[j]:
                Counter +=1
        if Counter >0:
            if Hum_TAD_Compare.get(i):
                Hum_TAD_Compare[i].append([j,Counter])
            else:
                Hum_TAD_Compare[i]=[[[j,Counter]]]

In [ ]:
Hum_TAD_Compare